In [1]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep

In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'

URL_FIRST_PAGE_HH = 'https://almaty.hh.kz/search/vacancy?text=python&area=160'

In [3]:
headers = {
    'User-Agent': USER_AGENT,
}

In [4]:
def parse_hh(url_page, headers, result=[], index_page=1):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
      print('Парсинг завершен')
      return result
    else:
      print('Cтраница №%d, ссылка: %s'%(index_page, response.url))

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
      result.append(parse_vacancy_hh(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
      link_next_page =  'https://almaty.hh.kz' + link_next_page['href']
    else:
      print('Парсинг завершен')
      return result

    result = parse_hh(link_next_page, headers, result, index_page+1)   
    return result
     
def parse_vacancy_hh(dom_vacancy):
  sleep(0.1)
  vacancy_name = dom_vacancy.find('a').text

  vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
  if vacancy_salary:
    vacancy_salary = vacancy_salary.text
    min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
  else:
    min_salary, max_salary, currency_salary = None, None, None

  vacancy_link = dom_vacancy.find('a')['href']
  
  return {
      'vacancy_name': vacancy_name,
      'vacancy_salary': vacancy_salary,
      'min_salary': min_salary,
      'max_salary': max_salary,
      'currency_salary': currency_salary,
      'vacancy_link': vacancy_link,
      'vacancy_source': 'almaty.hh.kz',
  }

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [5]:
result = parse_hh(URL_FIRST_PAGE_HH, headers)

Cтраница №1, ссылка: https://almaty.hh.kz/search/vacancy?text=python&area=160
Cтраница №2, ссылка: https://almaty.hh.kz/search/vacancy?text=python&area=160&page=1&hhtmFrom=vacancy_search_list
Cтраница №3, ссылка: https://almaty.hh.kz/search/vacancy?text=python&area=160&page=2&hhtmFrom=vacancy_search_list
Cтраница №4, ссылка: https://almaty.hh.kz/search/vacancy?text=python&area=160&page=3&hhtmFrom=vacancy_search_list
Парсинг завершен


In [6]:
len(result)

80

In [7]:
!apt install mongodb > log

In [8]:
!service mongodb start

 * Starting database mongodb
   ...done.


In [9]:
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from pymongo import MongoClient

In [11]:
client = MongoClient()

In [12]:
db = client.vacancies_python_hh

In [13]:
collection_vacancies_hh_kz = db.hh_kz

In [14]:
def check_and_save_vacancies_in_db(vacancies):
  for vacancy in vacancies:
    if not len(list(collection_vacancies_hh_kz.find({'vacancy_link': vacancy['vacancy_link']}))):
      collection_vacancies_hh_kz.insert_one(vacancy)

In [15]:
check_and_save_vacancies_in_db(result)

In [16]:
result_find = list(collection_vacancies_hh_kz.find())

In [17]:
len(result_find)

80